In [1]:
import csbsingleenv

import sys
# Assuming your Python file is in 'src' under 'my_project' directory
sys.path.append('../..')
import env_customizer

2024-05-09 12:13:33,859	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


In [2]:
import gymnasium as gym
from csbsingleenv import *
import time
import numpy as np
from ray.rllib.algorithms.algorithm import Algorithm
import csbsingleenv
import ray
import env_customizer

In [3]:
if ray.is_initialized():
  ray.shutdown()

runtime_env = {"py_modules": ["../.."]}
ray.init(runtime_env=runtime_env)

2024-05-09 12:13:36,972	INFO worker.py:1642 -- Started a local Ray instance.
2024-05-09 12:13:37,062	INFO packaging.py:518 -- Creating a file package for local directory '../..'.
2024-05-09 12:13:37,144	INFO packaging.py:346 -- Pushing file package 'gcs://_ray_pkg_549991fb1282f84a.zip' (9.61MiB) to Ray cluster...
2024-05-09 12:13:37,190	INFO packaging.py:359 -- Successfully pushed file package 'gcs://_ray_pkg_549991fb1282f84a.zip'.


Python version:,3.10.12
Ray version:,2.7.0


In [4]:
# path_to_checkpoint = "/home/healthcare/Desktop/Tristan/rl-projects/results/PPO_2024-05-07_11-30-42/PPO_CustomizedEnvironment_e91b3_00000_0_2024-05-07_11-30-42/checkpoint_000032"
path_to_checkpoint = "/home/healthcare/Desktop/Tristan/rl-projects/results/PPO_2024-05-09_11-56-53/PPO_CustomizedEnvironment_e6b26_00000_0_2024-05-09_11-56-53/checkpoint_000014"
algo = Algorithm.from_checkpoint(path_to_checkpoint)

2024-05-09 12:13:38,283	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.


2024-05-09 12:13:38,285	WARNING algorithm_config.py:672 -- Cannot create PPOConfig from given `config_dict`! Property __stdout_file__ not supported.
2024-05-09 12:13:38,288	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
/home/healthcare/.local/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py:484: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/home/healthcare/.local/lib/python3.

In [5]:
import numpy as np
import gymnasium as gym
from gymnasium import spaces, logger

# Note, this is only meant as an example of how to write a custom environment.
# I doubt it'll actually learn well.
class CustomizedRacing:
    def __init__(self):
        min_pos = -200000.0
        max_pos = 200000.0
        min_vel = -2000.0
        max_vel = 2000.0
        screen_max = [16000,9000]
        # This observation space will only have the next checkpoint rather than the next 5.
        self.observation_space = spaces.Box(
            low=np.array([0, -np.pi, min_pos, min_pos, min_vel, min_vel]+[0,0]),
            high=np.array([3, np.pi, max_pos, max_pos, max_vel, max_vel]+screen_max),
            dtype=np.float64
        )

        # This action will be how much to turn, rather than a point to turn towards.
        self.action_space = spaces.Box(
            low = np.array([-1.0, 0.0]),
            high = np.array([1.0, 100]),
            dtype=np.float64
        )

    # We can define and reset any extra variables we want to keep track of here.
    def reset(self):
        self.last_checkpoint = None
        self.last_sqr_dist = None

    def reward(self, env, rew):
        # Add reward for getting closer to the next checkpoint
        cur_dist = csbsingleenv.get_sqr_distance(env.pos, env.next_checkpoint())
        if self.last_sqr_dist is not None and cur_dist < self.last_sqr_dist:
            rew += 50
        self.last_sqr_dist = cur_dist
        
        # Add reward for passing a checkpoint
        cur_checkpoint = env.checkpoint_index
        if self.last_checkpoint is not None and cur_checkpoint != self.last_checkpoint:
            rew += 200
        self.last_checkpoint = cur_checkpoint
        
        if env.failed:
            return -1000
        return rew
    
    # Need to define the traformation.  Should be easy here since we just need to drop the excess data
    def transform_observation(self, env, obs):
        return obs[:8]

    # Need to also transform the angle change into a point to aim at.
    # Will define it as 1000 distance from the angle relative to current pointed direction
    def transform_action(self, env, action):
        preferred_dir = env.theta + action[0]*env.maxSteeringAngle
        dir_vector = 1000*csbsingleenv.angle_to_vector(preferred_dir)
        point_to_aim_at = env.pos + dir_vector
        trans_action = np.array([point_to_aim_at.x, point_to_aim_at.y, action[1]])
        return trans_action
    

In [6]:
env = env_customizer.CustomizedEnvironment(
    env_config={"env":csbsingleenv.CodersStrikeBackSingle(),
                "customizer":CustomizedRacing()})
fps =  env.metadata.get('video.frames_per_second')

# Set pseudorandom seed for the getting the same game
# env.seed(1234)
rewards = {}
for n in range(100):
    state, _ = env.reset()

    for i in range(1,100000):
        # display the game
        #render = env.render()
        #if not render:
        #    break
        # Take action (simple policy)
        targetX, targetY = state[6:8]
        thrust = 100
        action = algo.compute_single_action(state)
        #action = np.array([targetX, targetY, thrust])
        # Do a game step
        state, reward, done, trunc, _ = env.step(action)

        # Print the state

        print('---------- Tick %d' % i)
        print('Pod angle ', state[1])
        print('Pod (x,y): (%d, %d)' % (state[2], state[3]))
        print('Pod velocity (v_x,v_y): (%d, %d)' % (state[4], state[5]))
        print('First Checkpoint (x,y): (%d, %d)' % (state[6],state[7]))
        #print('Second Checkpoint (x,y): (%d, %d)' % (state[7],state[8]))
        print('Reward ', reward)
        rewards[reward] = rewards.get(reward, 0) + 1
        #print('Total Time', env.time)
        print('done ', done)

        # Slow down the cycle for a realistic simulation
        #time.sleep(1.0/fps)

        # The game end if the flag done is True
        if done:
            break
env.close()
rewards

2024-05-09 12:13:50,300	ERROR checker.py:258 -- Exception 'state_in' raised on function call without checkin input specs. RLlib will now attempt to check the spec before calling the function again.


SpecCheckingError: input spec validation failed on TorchStatefulActorCriticEncoder.forward, The data dict does not match the model specs. Keys ('state_in',) are in the spec dict but not on the data dict. Data keys are {('obs',)}.